ELS 엔진 클래스 (GBM + 할인 + 가격 + 델타)

In [2]:
import numpy as np
import pandas as pd

# ============================================================
# 1) ELS2Star4838 엔진
#    - 2자산 GBM
#    - 일별 sigma, rho, (r-q), 제로커브를 받아서
#      오늘 기준 가격/델타 계산
# ============================================================

class ELS2Star4838:
    def __init__(self,
                 T=3.0,
                 gap=0.5,
                 step_down=(0.92, 0.90, 0.85, 0.85, 0.80),
                 redemp_pay=(1.0275, 1.0550, 1.0825, 1.1100, 1.1375),
                 maturity_pay=1.1650,
                 ki=0.60,
                 sigma=None,           # [sigma_kospi, sigma_samsung]
                 rho=0.7477,
                 drift=None,           # [r_q_kospi, r_q_samsung]
                 S0=(1.0, 1.0),
                 zero_T=None,          # 제로커브 만기 (년 단위, ex. [0.25,0.5,1,2,3])
                 zero_r=None,          # 각 만기 제로금리 (연율, 소수; ex. 0.01 = 1%)
                 n_paths=100000,
                 seed=42):

        self.T = T
        self.gap = gap
        self.obs = np.arange(gap, T + 1e-12, gap)[:len(step_down)]

        self.step_down   = np.array(step_down)
        self.redemp_pay  = np.array(redemp_pay) * 10000
        self.maturity_pay = maturity_pay * 10000
        self.ki = ki

        self.sigma = np.array(sigma, dtype=float)
        self.rho   = float(rho)
        self.drift = np.array(drift, dtype=float)
        self.S0    = np.array(S0, dtype=float)

        # 공분산 → Cholesky
        corr = np.array([[1.0, self.rho],
                         [self.rho, 1.0]])
        D   = np.diag(self.sigma)
        cov = D @ corr @ D
        self.L = np.linalg.cholesky(cov)

        # 제로커브
        self.zero_T = np.array(zero_T, dtype=float)   # [0.25,0.5,1,2,3] 이런 식
        self.zero_r = np.array(zero_r, dtype=float)   # 각 만기 제로금리 (소수)

        self.n_paths = n_paths
        self.rng = np.random.default_rng(seed)

    # ------------------------------------------
    # 제로커브 기반 할인함수 DF(0→tau)
    # log DF를 선형보간 (log-linear DF)
    # ------------------------------------------
    def discount(self, tau):
        tau = np.asarray(tau, dtype=float)
        # 노드에서의 log DF
        log_df_nodes = -self.zero_r * self.zero_T     # r * T
        # tau에 대해 보간
        log_df_tau = np.interp(tau, self.zero_T, log_df_nodes)
        return np.exp(log_df_tau)

    # ------------------------------------------
    # GBM 시뮬레이션 (drift = r-q)
    # ------------------------------------------
    def _simulate(self, S0, T_remain, steps_per_year=252):
        dt = 1.0 / steps_per_year
        n_steps = int(T_remain * steps_per_year)

        paths = np.zeros((self.n_paths, n_steps + 1, 2))
        paths[:, 0, :] = S0

        drift_term = (self.drift - 0.5 * self.sigma**2) * dt
        sdt = np.sqrt(dt)

        Z  = self.rng.standard_normal((self.n_paths, n_steps, 2))
        dW = Z @ self.L.T    # 상관 구조 입힌 Brownian 증분 (단위분산)

        for t in range(n_steps):
            incr = np.exp(drift_term + sdt * dW[:, t, :])
            paths[:, t+1, :] = paths[:, t, :] * incr

        return paths

    # ------------------------------------------
    # 가격 계산 (오늘 t0 기준)
    # ------------------------------------------
    def price(self, t0, S0_current, past_min, steps_per_year=252):
        """
        t0         : 발행 후 경과연수 (예: 1.0 = 1년)
        S0_current : (S1/S1_start, S2/S2_start)
        past_min   : (과거 min1/S1_start, min2/S2_start)
        """
        T_remain = self.T - t0
        if T_remain <= 0:
            raise ValueError("t0 must be earlier than maturity.")

        S0_current = np.array(S0_current, dtype=float)
        past_min   = np.array(past_min, dtype=float)

        # 남은 조기상환 관측일
        remain_obs = self.obs[self.obs > t0]
        obs_tau = remain_obs - t0   # "오늘" 기준 남은 기간

        # 경로 시뮬레이션
        paths = self._simulate(S0_current, T_remain, steps_per_year)
        # shape: (n_paths, n_steps+1, 2)
        n = len(paths)

        # 과거/미래 최소값 합쳐서 녹인 여부 계산
        future_min = paths.min(axis=1).min(axis=1)     # 각 경로별 두 자산 전체기간 최솟값
        total_min  = np.minimum(future_min, np.min(past_min))
        ki_touch   = total_min < self.ki

        payoff   = np.zeros(n)
        redeemed = np.zeros(n, dtype=bool)

        # ----- 남은 조기상환일 체크 -----
        # obs_idx: 시뮬레이션 시간 index
        obs_idx = ((remain_obs - t0) * steps_per_year).astype(int)

        for i, idx in enumerate(obs_idx):
            # 아직 상환 안된 경로 중 배리어 위에 있는 것
            cond = (~redeemed) & np.all(paths[:, idx, :] >= self.step_down[i + len(self.obs) - len(remain_obs)], axis=1)
            if not np.any(cond):
                continue

            df = self.discount(obs_tau[i])
            payoff[cond]   = self.redemp_pay[i + len(self.obs) - len(remain_obs)] * df
            redeemed[cond] = True

        # ----- 만기 처리 (조기상환 안된 경로) -----
        not_red = ~redeemed
        if np.any(not_red):
            worst_T = paths[not_red, -1, :].min(axis=1)    # 만기 worst-of
            ki_hit  = ki_touch[not_red]

            p = np.zeros_like(worst_T)

            # 1) worst_T >= 0.80 : 쿠폰 지급
            p[worst_T >= 0.80] = self.maturity_pay

            # 2) worst_T < 0.80 이지만 KI 미터치 : 쿠폰 그대로 (설계별로 조정 가능)
            p[(worst_T < 0.80) & (~ki_hit)] = self.maturity_pay

            # 3) worst_T < 0.80 이고 KI 터짐 : 원금 × worst_T
            p[(worst_T < 0.80) & (ki_hit)] = 10000 * worst_T[(worst_T < 0.80) & (ki_hit)]

            df_T = self.discount(T_remain)
            payoff[not_red] = p * df_T

        # Monte Carlo 평균/표준오차
        price_mean = payoff.mean()
        price_se   = payoff.std(ddof=1) / np.sqrt(n)

        return price_mean, price_se

    # ------------------------------------------
    # 델타 (기초자산 2개 각각에 대해 중앙차분)
    # ------------------------------------------
    def delta(self, t0, S0_current, past_min, h=0.01, steps_per_year=252):
        S1, S2 = S0_current

        # 공통 난수 쓰기 위해 RNG 보존
        original_rng = self.rng
        self.rng = np.random.default_rng(12345)

        # Δ1 (KOSPI)
        V_up,  _ = self.price(t0, (S1*(1+h), S2), past_min, steps_per_year)
        V_dn,  _ = self.price(t0, (S1*(1-h), S2), past_min, steps_per_year)
        delta1 = (V_up - V_dn) / (2 * S1 * h)

        # Δ2 (Samsung)
        V_up2, _ = self.price(t0, (S1, S2*(1+h)), past_min, steps_per_year)
        V_dn2, _ = self.price(t0, (S1, S2*(1-h)), past_min, steps_per_year)
        delta2 = (V_up2 - V_dn2) / (2 * S2 * h)

        # RNG 원상복구
        self.rng = original_rng

        return delta1, delta2


ELS_Sheet1에서 데이터 읽고, 일별 가격+델타 계산

In [ ]:
# ============================================================
# 2) 데이터 로딩 및 전처리
# ============================================================

# 1) csv파일 읽기
df = pd.read_csv("ELS_Sheet1.csv")
df['date'] = pd.to_datetime(df['date'])

# 숫자로 변환해야 되는 컬럼들 
num_cols = [
    'samsung', 'kospi200',
    'vol_samsung', 'vol_kospi',
    'corr',
    'zero_rate_3M', 'zero_rate_6M', 'zero_rate_1Y',
    'zero_rate_2Y', 'zero_rate_3Y'
]

for col in num_cols:
    if col in df.columns:
        df[col] = (
            df[col]
            .astype(str)
            .str.replace(',', '')   # 쉼표 제거
            .str.strip()
            .replace('', np.nan)
            .astype(float)
        )

# 발행일 이후 필터링
issue_date = pd.to_datetime("2021-10-22")
filtered_df = df[df['date'] >= issue_date].reset_index(drop=True)

# 제로커브 관련 컬럼
zero_rate_cols = ['zero_rate_3M', 'zero_rate_6M', 'zero_rate_1Y',
                  'zero_rate_2Y', 'zero_rate_3Y']
horizons = np.array([0.25, 0.5, 1.0, 2.0, 3.0])   # 년 단위

# 발행일 기준 각 기초자산의 시작가격 가져오기
base_row = filtered_df.iloc[0]
samsung_start = base_row['samsung']
kospi_start   = base_row['kospi200']

results = []


# ------------------------------------------------------------
# 관측일(조기상환일) 리스트 생성
# ------------------------------------------------------------
obs_years = np.array([0.5, 1.0, 1.5, 2.0, 2.5])  # ELS 관측 시점
autocall_dates = issue_date + pd.to_timedelta((obs_years * 365).astype(int), unit='D')
autocall_dates = autocall_dates.round("D")  # 날짜 반올림

# ============================================================
# 3) Date별로 ELS 가격 + 델타 계산
#    - i=1부터: 발행일 다음날부터 평가/헤지
# ============================================================

filtered_df['year_month'] = filtered_df['date'].dt.to_period('M')
month_index_map = {}
for ym, grp in filtered_df.groupby('year_month'):
    idxs = grp.index.tolist()
    if not idxs:
        continue
    month_index_map[str(ym)] = (min(idxs), max(idxs))

print('month_index_map:', month_index_map)

for i in range(1, min(5, len(filtered_df))):
    row    = filtered_df.iloc[i]
    date_i = row['date']

    # ---- (1) 오늘 기준 파라미터 세팅 ----
    # 1) 변동성 (연율, 이미 소수라고 가정)
    sigma_i = np.array([row['vol_kospi'], row['vol_samsung']])

    # 2) 상관계수
    rho_i = row['corr']

    # 3) drift = r - q (지금은 0으로 둠. 나중에는 df 컬럼에서 가져오면 됨)
    drift_i = np.array([0.0, 0.0])

    # 4) 제로커브 (엑셀에 0.8045 같은 값이 "0.8045%"라면 /100 필요)
    zero_i = row[zero_rate_cols].to_numpy(dtype=float) / 100.0

    # ---- (2) 모델 생성 (오늘 기준 보정된 모형) ----
    model = ELS2Star4838(
        sigma   = sigma_i,
        rho     = rho_i,
        drift   = drift_i,
        zero_T  = horizons,
        zero_r  = zero_i,
        n_paths = 100_000,
        seed    = 42
    )

    # ---- (3) 오늘 기준 상태 세팅 ----
    samsung_cur = row['samsung']
    kospi_cur   = row['kospi200']

    # 발행일부터 오늘까지의 최소값 (녹인 체크용)
    samsung_min = filtered_df.loc[:i, 'samsung'].min()
    kospi_min   = filtered_df.loc[:i, 'kospi200'].min()

    # 상대가격 (발행일 = 1.0)
    S0_current = (
        kospi_cur   / kospi_start,
        samsung_cur / samsung_start
    )

    past_min = (
        kospi_min   / kospi_start,
        samsung_min / samsung_start
    )

    # 발행 후 경과연수
    t0 = (date_i - issue_date).days / 365.0

    # ------------------------------------------------------------
    # (A) 오늘이 조기상환일이면 -> 실제 조기상환 여부 확인 후 break
    # ------------------------------------------------------------
    if any(date_i == d for d in autocall_dates):

        # 조기상환 레벨 index 찾기
        level_idx = list(autocall_dates).index(date_i)
        barrier   = model.step_down[level_idx]  # 해당 시점의 barrier

        # 실제 조기상환 조건 충족?
        cond_real = (S0_current[0] >= barrier) and (S0_current[1] >= barrier)

        if cond_real:
            # 조기상환 지급 금액(할인 포함)
            tau_obs = obs_years[level_idx] - t0
            payoff_today = model.redemp_pay[level_idx] * model.discount(tau_obs)

            results.append({
                "date": date_i,
                "price": payoff_today,
                "price_se": 0.0,
                "delta_kospi": 0.0,
                "delta_samsung": 0.0
            })

            print(f"조기상환 발생! 날짜={date_i}, 지급액={payoff_today:.4f}")
            break  # ★ 루프 종료 ★

    # ------------------------------------------------------------
    # (B) 조기상환이 아니면 기존처럼 가격/델타 계산
    # ------------------------------------------------------------
    price_mean, price_se = model.price(t0, S0_current, past_min)
    delta_kospi, delta_samsung = model.delta(t0, S0_current, past_min)

    # ====== 시장 델타(market delta) 변환 ======
    delta_kospi_mkt   = delta_kospi   / kospi_start
    delta_samsung_mkt = delta_samsung / samsung_start

    # ====== 실제 헷지 수량 ======
    # 삼성전자: 시장 델타 = 주식 1원 변동당 ELS 가격변화 → 헷지 주식 수와 동일
    hedge_shares_samsung = delta_samsung_mkt

    # 코스피200: 선물 1pt = 250,000원 (승수)
    hedge_contracts_kospi = delta_kospi_mkt / 250000.0

    results.append({
        "date": date_i,
        "price": price_mean,
        "price_se": price_se,
        "delta_kospi_norm": delta_kospi,
        "delta_samsung_norm": delta_samsung,
        "delta_kospi_mkt": delta_kospi_mkt,
        "delta_samsung_mkt": delta_samsung_mkt,
        "hedge_contracts_kospi": hedge_contracts_kospi,
        "hedge_shares_samsung": hedge_shares_samsung
    })


# 리스트 → DataFrame
results_df = pd.DataFrame(results)
print(results_df.head())


month_index_map: {'2021-10': (0, 5), '2021-11': (6, 27), '2021-12': (28, 49), '2022-01': (50, 69), '2022-02': (70, 87), '2022-03': (88, 108), '2022-04': (109, 129), '2022-05': (130, 150), '2022-06': (151, 170), '2022-07': (171, 191), '2022-08': (192, 213), '2022-09': (214, 233), '2022-10': (234, 252), '2022-11': (253, 274), '2022-12': (275, 295), '2023-01': (296, 315), '2023-02': (316, 335), '2023-03': (336, 357), '2023-04': (358, 377), '2023-05': (378, 397), '2023-06': (398, 418), '2023-07': (419, 439), '2023-08': (440, 461), '2023-09': (462, 480), '2023-10': (481, 499), '2023-11': (500, 521), '2023-12': (522, 540), '2024-01': (541, 562), '2024-02': (563, 581), '2024-03': (582, 601), '2024-04': (602, 622), '2024-05': (623, 642), '2024-06': (643, 661), '2024-07': (662, 684), '2024-08': (685, 705), '2024-09': (706, 723), '2024-10': (724, 741)}


'\nfor i in range(1, min(5, len(filtered_df))):\n    row    = filtered_df.iloc[i]\n    date_i = row[\'date\']\n\n    # ---- (1) 오늘 기준 파라미터 세팅 ----\n    # 1) 변동성 (연율, 이미 소수라고 가정)\n    sigma_i = np.array([row[\'vol_kospi\'], row[\'vol_samsung\']])\n\n    # 2) 상관계수\n    rho_i = row[\'corr\']\n\n    # 3) drift = r - q (지금은 0으로 둠. 나중에는 df 컬럼에서 가져오면 됨)\n    drift_i = np.array([0.0, 0.0])\n\n    # 4) 제로커브 (엑셀에 0.8045 같은 값이 "0.8045%"라면 /100 필요)\n    zero_i = row[zero_rate_cols].to_numpy(dtype=float) / 100.0\n\n    # ---- (2) 모델 생성 (오늘 기준 보정된 모형) ----\n    model = ELS2Star4838(\n        sigma   = sigma_i,\n        rho     = rho_i,\n        drift   = drift_i,\n        zero_T  = horizons,\n        zero_r  = zero_i,\n        n_paths = 100_000,\n        seed    = 42\n    )\n\n    # ---- (3) 오늘 기준 상태 세팅 ----\n    samsung_cur = row[\'samsung\']\n    kospi_cur   = row[\'kospi200\']\n\n    # 발행일부터 오늘까지의 최소값 (녹인 체크용)\n    samsung_min = filtered_df.loc[:i, \'samsung\'].min()\n    kospi_min   = filtered_d